### Classes More Than 5 Students

In [4]:
import pandas as pd
import pandasql as ps

data = [['A', 'Math'], ['B', 'English'], ['C', 'Math'], ['D', 'Biology'], ['E', 'Math'], ['F', 'Computer'], ['G', 'Math'], ['H', 'Math'], ['I', 'Math']]
Courses = pd.DataFrame(data, columns=['student', 'class']).astype({'student':'object', 'class':'object'})
print(Courses)

  student     class
0       A      Math
1       B   English
2       C      Math
3       D   Biology
4       E      Math
5       F  Computer
6       G      Math
7       H      Math
8       I      Math


In [112]:
result = Courses.groupby('class').count().reset_index()
print(result[result['student']>=5][['class']].reset_index(drop=True))

  class
0  Math


In [111]:
query = '''
select class
from Courses
group by class
having count(*) > 5
'''

print(ps.sqldf(query,locals()))

  class
0  Math


### Customer Placing the Largest Number of Orders

In [11]:
data = [[1, 1], [2, 2], [3, 3], [4, 3]]
orders = pd.DataFrame(data, columns=['order_number', 'customer_number']).astype({'order_number':'Int64', 'customer_number':'Int64'})
print(orders)

   order_number  customer_number
0             1                1
1             2                2
2             3                3
3             4                3


In [42]:
print(orders.groupby('customer_number')[['order_number']].count().nlargest(1, 'order_number').reset_index()[['customer_number']])

   customer_number
0                3


In [47]:
query = '''
select customer_number
from orders
group by customer_number
order by count(order_number) desc
limit 1
'''

print(ps.sqldf(query,locals()))

   customer_number
0                3


### Group Sold Products By The Date

In [48]:
data = [['2020-05-30', 'Headphone'], ['2020-06-01', 'Pencil'], ['2020-06-02', 'Mask'], ['2020-05-30', 'Basketball'], ['2020-06-01', 'Bible'], ['2020-06-02', 'Mask'], ['2020-05-30', 'T-Shirt']]
Activities = pd.DataFrame(data, columns=['sell_date', 'product']).astype({'sell_date':'datetime64[ns]', 'product':'object'})
print(Activities)

   sell_date     product
0 2020-05-30   Headphone
1 2020-06-01      Pencil
2 2020-06-02        Mask
3 2020-05-30  Basketball
4 2020-06-01       Bible
5 2020-06-02        Mask
6 2020-05-30     T-Shirt


In [54]:
Activities.groupby('product').count()

,sell_date
product,
Basketball,1
Bible,1
Headphone,1
Mask,2
Pencil,1
T-Shirt,1


In [74]:
Activities.groupby('sell_date')['product'].agg([
        ('num_sold', 'nunique'),
        ('products', lambda x: ', '.join(sorted(x.unique())))
    ]).reset_index()

,sell_date,num_sold,products
0,2020-05-30,3,"Basketball, Headphone, T-Shirt"
1,2020-06-01,2,"Bible, Pencil"
2,2020-06-02,1,Mask


In [120]:
query_my = '''
select 
    date(sell_date) as sell_date, 
    count(distinct product) as num_sold, 
    group_concat(distinct product order by product asc separator ',') as products
from Activities
group by sell_date
order by sell_date asc;
'''

query_sql = '''
with OrderedActivities as (
    select sell_date, product
    from Activities
    order by sell_date, product
)

SELECT 
    date(sell_date) as sell_date, 
    count(distinct product) as num_products, 
    group_concat(distinct product) as products
from OrderedActivities
group by date(sell_date);

'''

print(ps.sqldf(query_sql,locals())) 

    sell_date  num_products                      products
0  2020-05-30             3  Basketball,Headphone,T-Shirt
1  2020-06-01             2                  Bible,Pencil
2  2020-06-02             1                          Mask
